In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
from gensim.models.fasttext import load_facebook_model
from tabulate import tabulate
import random

# **Pretrained Model**

In [8]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
! gunzip "cc.en.300.bin.gz"

/tmp/ipykernel_34/3772731771.py:5: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format(model_path)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_model/cc.en.300.bin'

In [4]:
tips_df = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True)
tips_df

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [6]:
texts=list(tips_df['text'])
sentences=texts[:1000]
print("\nSamples of Sentences\n {}".format(sentences[0:10]))


Samples of Sentences
 ['Avengers time with the ladies.', 'They have lots of good deserts and tasty cuban sandwiches', "It's open even when you think it isn't", 'Very decent fried chicken', 'Appetizers.. platter special for lunch', 'Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.', "Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!", 'This is probably the best place in the cool Springs area to watch a game and eat', 'Tacos', 'Starbucks substitute in boring downtown Tampa. Ugh. Never again!']


In [7]:
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
unzip:  cannot find or open /kaggle/working/nltk_data/corpora/wordnet.zip, /kaggle/working/nltk_data/corpora/wordnet.zip.zip or /kaggle/working/nltk_data/corpora/wordnet.zip.ZIP.
unzip:  cannot find or open /kaggle/working/nltk_data/corpora/omw-1.4.zip, /kaggle/working/nltk_data/corpora/omw-1.4.zip.zip or /kaggle/working/nltk_data/corpora/omw-1.4.zip.ZIP.


In [ ]:

def process_text(text):
    text = re.sub(r'\s+', ' ', text, flags=re.I) # Remove extra white space from text

    text = re.sub(r'\W', ' ', str(text)) # Remove all the special characters from text

    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # Remove all single characters from text

    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove any character that isn't alphabetical

    text = text.lower()

    words = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    stop_words = set(stopwords.words("english"))
    Words = [word for word in words if word not in stop_words]

    Words = [word for word in Words if len(word) > 3]

    indices = np.unique(Words, return_index=True)[1]
    cleaned_text = np.array(Words)[np.sort(indices)].tolist()

    return cleaned_text

In [ ]:
cleaned_text = [process_text(text) for text in sentences]

In [ ]:
print(cleaned_text[:5])

In [ ]:
def train_Fasttext(sentences,embedding_size,window_size,min_word,down_sampling,epochs,Save_model_filename):
    fast_Text_model = FastText(sentences,
    vector_size=embedding_size, # Dimensionality of the word vectors. ,
    window=window_size,
    min_count=min_word, # The model ignores all words with total frequency lower than this.
    sample=down_sampling, # threshold which higher-frequency words are randomly down sampled
    workers = 4, # Num threads to train the model (faster training with multicore comp.)
    sg=1, # Training algorithm: skip-gram if sg=1, otherwise CBOW.
    epochs=epochs) # Number of iterations (epochs) over the corpus

    fast_Text_model.save(Save_model_filename) # Save fastText gensim model

In [ ]:
embedding_size = 300
window_size = 3
min_word = 1
down_sampling = 1e-2
epochs=300

train_Fasttext(cleaned_text,embedding_size,window_size,min_word,down_sampling,epochs,"custom_fast_text")

In [ ]:
fast_Text_model = Word2Vec.load("/kaggle/working/custom_fast_text") 


In [ ]:
pretrained_fastText = load_facebook_model('/kaggle/working/cc.en.300.bin')

In [ ]:
! pip install reportlab

In [ ]:
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet

In [ ]:
def create_pdf(tables):
    doc = SimpleDocTemplate("Words.pdf", pagesize=letter)
    doc.build(tables)

In [ ]:
words = list(fast_Text_model.wv.key_to_index)
tables = []
c=0
for _ in range(20):
    data = []
    word = random.choice(words)
    words.remove(word)
    c += 1
    print(f"word: {word}\n")

    data.append(['word: ' + word])
    ######## custom model ########
    similar_words_custom = fast_text_model.wv.most_similar(positive=[word], topn=10)
    opposite_words_custom = fast_text_model.wv.most_similar(negative=[word], topn=10)

    data.append(['Top 10 similar words (custom model):'])
    data.append(['Similar Word', 'Similarity'])
    data.extend(similar_words_custom)
    data.append([''])

    data.append(['Top 10 dissimilar words (custom model):'])
    data.append(['Dissimilar Word', 'Dissimilarity'])
    data.extend(opposite_words_custom)
    data.append([''])

    ######## pretrained model ########
    similar_words_pretrained = pretrained_facebook.wv.most_similar(positive=[word], topn=10)
    opposite_words_pretrained = pretrained_facebook.wv.most_similar(negative=[word], topn=10)

    data.append(['Top 10 similar words (pre-trained model):'])
    data.append(['Similar Word', 'Similarity'])
    data.extend(similar_words_pretrained)
    data.append([''])

    data.append(['Top 10 dissimilar words (pre-trained model):'])
    data.append(['Dissimilar Word', 'Dissimilarity'])
    data.extend(opposite_words_pretrained)
    data.append([''])

    table = Table(data)

    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 14),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 12),
        ('LEFTPADDING', (0, 1), (-1, -1), 12),
        ('RIGHTPADDING', (0, 1), (-1, -1), 12),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 12),
    ]))

    tables.append(table)
    tables.append(Spacer(1, 20))


    # Creating tables for each category
    table_custom_similar = tabulate(similar_words_custom, headers=['Similar Word', 'Similarity'], tablefmt='github')
    table_custom_opposite = tabulate(opposite_words_custom, headers=['Opposite Word', 'Similarity'], tablefmt='github')
    table_pretrained_similar = tabulate(similar_words_pretrained, headers=['Similar Word', 'Similarity'], tablefmt='github')
    table_pretrained_opposite = tabulate(opposite_words_pretrained, headers=['Opposite Word', 'Similarity'], tablefmt='github')
    if c == 20:
        break
create_pdf(tables)
print("PDF Done !")